**Un correcteur de questions à choix multiples**

Les élèves

1.   utilisent obligatoirement une copie de [cette feuille-réponse](https://drive.google.com/file/d/1fs7slGV1wB0Djzp2CHAYDLdNK_kc6jpx/view?usp=drive_link).
2.   identifient leur copie à l'aide de leur identifiant réseau du CSSMB
1.   noirciront le cercle contenant la lettre de leur choix.

L'enseignant

1.   numérise les feuilles-réponses des élèves au photocopieur.

1.   renomme le fichier pdf: eleve.pdf

2.   [convertie le pdf](https://pdf2png.com/) en plusieurs png: ils seront nommés selon le format eleve-1.pdf, eleve-1.pdf...
1.   téléverse les fichiers png dans le sous dossier : Fichiers-Eleves




<img src="https://drive.google.com/uc?export=view&id=1NB1YrpqoLT4va5Z1o3Gv3r1FksDjmnRg" alt="Description de l'image" width="66%">


Passer votre souris sur le [ ], puis cliquer sur le ▷ pour exécuter le code.

Google vérifiera qu'il s'agit de toi! Accepte les demandes d'accès de ton code à ton dossier.


In [ ]:
# Cellule 01
# Importer les librairies de codes nécessaires
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import glob  #
import re    # Pour le tri naturel des noms de fichiers

# Le code de cette cellule est utile seulement pour utiliser les données dans un environnement Google Colab.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

**Configure les chemins et le fichier de Sortie**
1. Créer un sous-dossier et le nommer Fichiers-Eleves.
2. Le programme est configuré pour résider dans *Mon disque* et le dossier *Colab Notebooks* qui contient un dossier *Correcteur-QCM* qui contient à son tour un *dossier* *Fichiers-Eleves*.
3. Téléverser les images *eleve-x.png* dans le sous-dossier Fichiers-Eleves.

Tu peux certainement modifier le code de la prochaine cellule pour faire correspondre les chemin avec la structure de ton *Mon drive*.



In [ ]:
# Cellule 02: Chemins et Fichier de Sortie (Configuration)

input_folder = '/content/drive/MyDrive/Colab Notebooks/Correcteur-QCM/Fichiers-Eleves'  # Chemin du dosier dans lequel les images sont téléversées
output_csv = '/content/drive/MyDrive/Colab Notebooks/Correcteur-QCM/resultats.csv' # Nom du CSV pour le lot




*   S'il y a des erreurs de lecture de cercles noicis, il faut modifier les paramètres de la cellule suivante.
*   Si tu veux utiliser ton propre gabarit de feuille-réponse, tu dois mesurer les positions des cercles en pixels. J'utilise tout simplement MS Paint.

In [ ]:
# Cellule 03: Paramètres pour la détection de l'ID et du QCM

# --- Paramètres pour la détection de l'ID ---
ID_Y_COORDS = [311, 392, 475, 557, 638, 719, 802]
ID_X_START_LETTER = 219
ID_X_START_DIGIT = 219
LETTER_SPACING = 44
DIGIT_SPACING = 44
NUM_LETTERS_OPTIONS = 26  # A-Z
NUM_DIGITS_OPTIONS = 10   # 0-9
ID_MATCHING_THRESHOLD = 15 # Seuil pour associer un cercle noirci à une position d'ID
ID_VERTICAL_ROW_TOLERANCE = (ID_Y_COORDS[1] - ID_Y_COORDS[0]) / 2.5 if len(ID_Y_COORDS) > 1 else 10

# --- Paramètres pour la détection des réponses QCM ---
QCM_NUM_QUESTIONS = 15      # Nombre de lignes (questions) dans la feuille de réponse QCM
QCM_X_START = 353           # Coordonnée x de départ pour la première colonne de réponses QCM (A)
QCM_Y_START = 960           # Coordonnée y de départ pour la première ligne de réponses QCM (Question 1)
QCM_ANSWER_COLUMN_WIDTH = 43 # Espacement horizontal entre les centres des choix (A-B, B-C, etc.)
QCM_ANSWER_ROW_HEIGHT = 60   # Espacement vertical entre les lignes de questions QCM
QCM_NUM_CHOICES = 4         # Nombre de choix par question (A, B, C, D)
QCM_MATCHING_THRESHOLD = 20 # Seuil de distance pour considérer un cercle comme proche d'une zone de réponse QCM (était threshold_value)

# --- Paramètres communs pour HoughCircles (utilisés pour détecter TOUS les cercles) ---
# Ajustez ces paramètres pour qu'ils détectent bien les cercles de l'ID ET du QCM.
# Si les tailles/types de cercles sont très différents, deux passes de HoughCircles pourraient être envisagées,
# mais essayons avec un seul jeu de paramètres d'abord.
HOUGH_DP = 1.2
HOUGH_MIN_DIST = 20      # Distance minimale entre les centres des cercles détectés.
HOUGH_PARAM1 = 50        # Seuil supérieur pour le détecteur de contours Canny.
HOUGH_PARAM2 = 25        # Seuil pour la détection des centres. Plus il est petit, plus il détecte de cercles (y compris des faux).
HOUGH_MIN_RADIUS = 8
HOUGH_MAX_RADIUS = 15    # Ajustez en fonction de la taille des plus petits et plus grands cercles sur la feuille.

Il y a peu de code à changer dans les cellules suivantes.

*   C'est ici que l'on peut modifier le diamètres des cercles de correction.
*   C'est ici que l'on change le nombre de lignes de l'identifiant.



In [ ]:
# Cellule 04: Définitions des Cartes de Positions et Fonctions (Configuration)

# --- Fonction pour l'intensité (pour l'ID et QCM) ---
def get_average_intensity(image_gray, center_x, center_y, radius):
    cx, cy, r = int(center_x), int(center_y), int(radius)
    mask = np.zeros(image_gray.shape, dtype=np.uint8)
    mask_radius = max(1, int(r * 0.7))
    cv2.circle(mask, (cx, cy), mask_radius, 255, -1)
    mean_val = cv2.mean(image_gray, mask=mask)[0]
    return mean_val

# --- Carte des positions des caractères de l'ID ---
id_character_positions_map = []
for i in range(6): # Lignes de lettres
    y = ID_Y_COORDS[i]
    row_char_positions = []
    for j in range(NUM_LETTERS_OPTIONS):
        x = ID_X_START_LETTER + j * LETTER_SPACING
        char_value = chr(65 + j)
        row_char_positions.append({'x': x, 'y': y, 'char': char_value, 'id_row_index': i, 'type': 'letter'})
    id_character_positions_map.append(row_char_positions)
y_digits_id = ID_Y_COORDS[6] # Ligne de chiffres ID
row_digit_positions_id = []
for j in range(NUM_DIGITS_OPTIONS):
    x = ID_X_START_DIGIT + j * DIGIT_SPACING
    char_value = str(j)
    row_digit_positions_id.append({'x': x, 'y': y_digits_id, 'char': char_value, 'id_row_index': 6, 'type': 'digit'})
id_character_positions_map.append(row_digit_positions_id)
print("Carte des positions pour l'ID créée.")

# --- Carte des positions des réponses QCM ---
qcm_answer_areas = []
for q_row in range(QCM_NUM_QUESTIONS):
    y_center_qcm = QCM_Y_START + q_row * QCM_ANSWER_ROW_HEIGHT
    q_row_answer_positions = []
    for choice_col in range(QCM_NUM_CHOICES):
        x_center_qcm = QCM_X_START + choice_col * QCM_ANSWER_COLUMN_WIDTH
        choice_char = chr(65 + choice_col)
        q_row_answer_positions.append({'x': x_center_qcm, 'y': y_center_qcm, 'char': choice_char,
                                       'question_idx': q_row, 'choice_idx': choice_col})
    qcm_answer_areas.append(q_row_answer_positions)
print(f"Carte des {QCM_NUM_QUESTIONS} questions QCM créée.")

# --- Clés pour le CSV de sortie ---
CSV_FIELDNAMES = ['ImageName', 'DetectedID'] + [f'Q{i+1}' for i in range(QCM_NUM_QUESTIONS)]

Carte des positions pour l'ID créée.
Carte des 15 questions QCM créée.


C'est dans cette cellule que vous pourrez modifier le nom des images à analyser: (eleve-1, eleve-2, ..., eleve-10). Il y a aussi du code pour aider à débogger.

In [ ]:
# Cellule 06: Traitement par Lots des Images

# Fonction pour trier naturellement les noms de fichiers (eleve-1, eleve-2, ..., eleve-10)
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

# Lister tous les fichiers images correspondants au modèle
image_files_pattern = os.path.join(input_folder, 'eleve-*.png')
list_of_image_paths = glob.glob(image_files_pattern)
list_of_image_paths.sort(key=natural_sort_key) # Trier les fichiers

if not list_of_image_paths:
    print(f"Aucun fichier image trouvé dans {input_folder} avec le modèle eleve-*.png")
else:
    print(f"{len(list_of_image_paths)} fichiers images à traiter.")

# Vérifier si le fichier CSV existe pour l'en-tête (une seule fois avant la boucle)
csv_file_already_exists = os.path.isfile(output_csv)

try:
    with open(output_csv, 'a' if csv_file_already_exists else 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=CSV_FIELDNAMES)
        if not csv_file_already_exists:
            writer.writeheader()

        # Boucle principale pour traiter chaque fichier image
        for image_path in list_of_image_paths:
            current_image_filename = os.path.basename(image_path)
            print(f"\n--- Traitement du fichier : {current_image_filename} ---")

            image_orig = cv2.imread(image_path)
            if image_orig is None:
                print(f"ERREUR : Impossible de lire l'image {current_image_filename}. Fichier ignoré.")
                # Écrire une ligne d'erreur dans le CSV pour ce fichier
                error_row = {'ImageName': current_image_filename, 'DetectedID': 'ERREUR LECTURE IMAGE'}
                for i in range(QCM_NUM_QUESTIONS): error_row[f'Q{i+1}'] = "N/A"
                writer.writerow(error_row)
                continue

            image_for_processing = image_orig.copy()

            # 1. Prétraitement et Détection Globale de Cercles
            gray = cv2.cvtColor(image_for_processing, cv2.COLOR_BGR2GRAY)
            blur = cv2.medianBlur(gray, 5)
            circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, dp=HOUGH_DP, minDist=HOUGH_MIN_DIST,
                                       param1=HOUGH_PARAM1, param2=HOUGH_PARAM2,
                                       minRadius=HOUGH_MIN_RADIUS, maxRadius=HOUGH_MAX_RADIUS)

            overall_detected_circles = []
            if circles is not None:
                circles_processed = np.uint16(np.around(circles))
                overall_detected_circles = sorted(circles_processed[0, :], key=lambda c: (c[1], c[0]))
                # print(f"{len(overall_detected_circles)} cercles bruts détectés pour {current_image_filename}.") # Optionnel
            else:
                print(f"Aucun cercle brut détecté pour {current_image_filename}.")

            # Initialisations pour les résultats de l'image actuelle
            final_id_string = "NO_ID_PROC"
            id_string_parts = [None] * 7
            qcm_answers_by_question_idx = {}

            # 2. Logique de Détection de l'ID
            if not overall_detected_circles:
                print(f"ID : Aucun cercle global pour {current_image_filename}.")
                final_id_string = "NO_CIRCLES_FOR_ID"
            else:
                for id_row_idx in range(7):
                    # ... (copier ici la logique de traitement d'une ligne d'ID de la Cellule 07 précédente)
                    # S'assurer d'utiliser 'overall_detected_circles' et 'ID_VERTICAL_ROW_TOLERANCE' etc.
                    # Le code ci-dessous est une version abrégée de cette logique
                    predefined_chars_for_id_row = id_character_positions_map[id_row_idx]
                    current_id_row_y_ref = int(predefined_chars_for_id_row[0]['y'])
                    id_circles_near_this_row = [c for c in overall_detected_circles if abs(int(c[1]) - current_id_row_y_ref) < float(ID_VERTICAL_ROW_TOLERANCE)]

                    if not id_circles_near_this_row: continue
                    darkest_id_circle_on_row = None; min_id_intensity = float('inf')
                    if len(id_circles_near_this_row) == 1: darkest_id_circle_on_row = id_circles_near_this_row[0]
                    else:
                        for id_c_coords in id_circles_near_this_row:
                            avg_id_intensity = get_average_intensity(gray, int(id_c_coords[0]), int(id_c_coords[1]), int(id_c_coords[2]))
                            if avg_id_intensity < min_id_intensity: min_id_intensity = avg_id_intensity; darkest_id_circle_on_row = id_c_coords

                    if darkest_id_circle_on_row is None: continue

                    marked_id_cx, marked_id_cy = int(darkest_id_circle_on_row[0]), int(darkest_id_circle_on_row[1])
                    assoc_char_id = None; min_dist_id = float('inf')
                    for id_char_pos in predefined_chars_for_id_row:
                        dist_id = np.sqrt(float(marked_id_cx - int(id_char_pos['x']))**2 + float(marked_id_cy - int(id_char_pos['y']))**2)
                        if dist_id < min_dist_id: min_dist_id = dist_id; assoc_char_id = id_char_pos['char']
                    if assoc_char_id is not None and min_dist_id <= ID_MATCHING_THRESHOLD: id_string_parts[id_row_idx] = assoc_char_id

                # Assemblage de l'ID final pour l'image actuelle
                is_id_complete = True; temp_id_str = ""
                for i in range(6): temp_id_str += id_string_parts[i] if id_string_parts[i] else "?"; is_id_complete &= (id_string_parts[i] is not None)
                temp_id_str += id_string_parts[6] if id_string_parts[6] else "?"; is_id_complete &= (id_string_parts[6] is not None)
                final_id_string = temp_id_str
                # print(f"ID pour {current_image_filename}: {final_id_string}") # Optionnel

            # 3. Logique de Détection QCM
            if not overall_detected_circles:
                print(f"QCM : Aucun cercle global pour {current_image_filename}.")
            else:
                # print(f"\nTraitement QCM pour {current_image_filename}...") # Optionnel
                for q_idx in range(QCM_NUM_QUESTIONS):
                    # ... (copier ici la logique de traitement d'une question QCM de la Cellule 09 précédente)
                    # S'assurer d'utiliser 'overall_detected_circles' et les bons paramètres QCM.
                    # Le code ci-dessous est une version abrégée de cette logique
                    current_q_y_ref = int(QCM_Y_START + q_idx * QCM_ANSWER_ROW_HEIGHT)
                    qcm_vertical_tolerance = float(QCM_ANSWER_ROW_HEIGHT / 3.0)
                    circles_for_this_q_row = [c for c in overall_detected_circles if abs(int(c[1]) - current_q_y_ref) < qcm_vertical_tolerance]

                    if not circles_for_this_q_row: qcm_answers_by_question_idx[q_idx] = "?"; continue
                    darkest_qcm_circle_on_row = None; min_qcm_intensity = float('inf')
                    if len(circles_for_this_q_row) == 1: darkest_qcm_circle_on_row = circles_for_this_q_row[0]
                    else:
                        for q_c_coords in circles_for_this_q_row:
                            avg_qcm_intensity = get_average_intensity(gray, int(q_c_coords[0]), int(q_c_coords[1]), int(q_c_coords[2]))
                            if avg_qcm_intensity < min_qcm_intensity: min_qcm_intensity = avg_qcm_intensity; darkest_qcm_circle_on_row = q_c_coords

                    if darkest_qcm_circle_on_row is None: qcm_answers_by_question_idx[q_idx] = "?"; continue

                    marked_qcm_cx, marked_qcm_cy = int(darkest_qcm_circle_on_row[0]), int(darkest_qcm_circle_on_row[1])
                    assoc_choice_char = None; min_dist_qcm = float('inf')
                    predefined_choices_for_this_q = qcm_answer_areas[q_idx]
                    for q_choice_pos in predefined_choices_for_this_q:
                        dist_qcm = np.sqrt(float(marked_qcm_cx - int(q_choice_pos['x']))**2 + float(marked_qcm_cy - int(q_choice_pos['y']))**2)
                        if dist_qcm < min_dist_qcm: min_dist_qcm = dist_qcm; assoc_choice_char = q_choice_pos['char']
                    if assoc_choice_char is not None and min_dist_qcm <= QCM_MATCHING_THRESHOLD: qcm_answers_by_question_idx[q_idx] = assoc_choice_char
                    else: qcm_answers_by_question_idx[q_idx] = "?"

                # Affichage des réponses QCM pour l'image actuelle (optionnel)
                # print(f"Réponses QCM pour {current_image_filename}:")
                # for i in range(QCM_NUM_QUESTIONS): print(f"  Question {i + 1}: {qcm_answers_by_question_idx.get(i, '?')}")


            # 4. Préparer la ligne de données et écrire dans le CSV
            csv_data_row = {'ImageName': current_image_filename, 'DetectedID': final_id_string}
            for i in range(QCM_NUM_QUESTIONS):
                csv_data_row[f'Q{i+1}'] = qcm_answers_by_question_idx.get(i, "?")

            writer.writerow(csv_data_row)
            print(f"Résultats pour {current_image_filename} sauvegardés.")

    print(f"\nTraitement par lots terminé. Résultats dans {output_csv}")

except IOError:
    print(f"ERREUR E/S lors de l'écriture dans le fichier CSV : {output_csv}")
except Exception as e:
    print(f"Une erreur générale est survenue : {e}")
    import traceback
    traceback.print_exc()

7 fichiers images à traiter.

--- Traitement du fichier : ID-1.png ---
Résultats pour ID-1.png sauvegardés.

--- Traitement du fichier : ID-2.png ---
Résultats pour ID-2.png sauvegardés.

--- Traitement du fichier : ID-3.png ---
Résultats pour ID-3.png sauvegardés.

--- Traitement du fichier : ID-4.png ---
Résultats pour ID-4.png sauvegardés.

--- Traitement du fichier : ID-5.png ---
Résultats pour ID-5.png sauvegardés.

--- Traitement du fichier : ID-6.png ---
Résultats pour ID-6.png sauvegardés.

--- Traitement du fichier : ID-7.png ---
Résultats pour ID-7.png sauvegardés.

Traitement par lots terminé. Résultats dans /content/drive/MyDrive/Colab Notebooks/Correcteur-QCM/resultats_complets_batch.csv


Définition des zones de réponse et tri des cercles

<img src="https://www.cssmb.gouv.qc.ca/wp-content/uploads/2021/10/LogoCSS-MentionAvenir_Couleur.png" width=30% align="left">